In [51]:
import numpy as np
import pandas as pd

In [52]:
df = pd.read_csv('diabetes.csv')

In [53]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [55]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [56]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [57]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
scaler = StandardScaler()

In [58]:
x = scaler.fit_transform(x)
x.shape

(768, 8)

In [59]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 ,random_state=2)

In [133]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout
import matplotlib.pyplot as plt

In [61]:
model = Sequential()

In [62]:
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=100, batch_size=32,validation_data=(x_test,y_test))

Epoch 1/100


/home/ashu/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3537 - loss: 0.8445 - val_accuracy: 0.3701 - val_loss: 0.7678
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4474 - loss: 0.7301 - val_accuracy: 0.5455 - val_loss: 0.7011
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6251 - loss: 0.6747 - val_accuracy: 0.6558 - val_loss: 0.6533
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6601 - loss: 0.6355 - val_accuracy: 0.6948 - val_loss: 0.6192
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7143 - loss: 0.5921 - val_accuracy: 0.7208 - val_loss: 0.5903
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7479 - loss: 0.5631 - val_accuracy: 0.7078 - val_loss: 0.5678
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7172 - loss: 0.5538 - val_accuracy: 0.7078 - val_loss: 0.5505
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7357 - loss: 0.5415 - val_accuracy: 0.7208 - val_loss: 0.5

## KERAS TUNER

In [64]:
import kerastuner as kt

In [78]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [79]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [80]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [81]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [82]:
model = tuner.get_best_models(num_models=1)[0]

In [83]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [84]:
model.fit(x_train,y_train,batch_size=32,epochs=200,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7061 - loss: 0.5654 - val_accuracy: 0.7727 - val_loss: 0.5022
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7342 - loss: 0.5433 - val_accuracy: 0.7662 - val_loss: 0.4914
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7538 - loss: 0.5248 - val_accuracy: 0.7662 - val_loss: 0.4830
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7351 - loss: 0.5262 - val_accuracy: 0.7792 - val_loss: 0.4754
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7582 - loss: 0.5111 - val_accuracy: 0.7792 - val_loss: 0.4700
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7474 - loss: 0.5022 - val_accuracy: 0.7792 - val_loss: 0.4663
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7533 - loss: 0.5123 - val_accuracy: 0.7857 - val_loss: 0.4633
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7861 - loss: 0.4746 - val_accuracy: 0.785

## HOW TO FIND RIGHT NUMBER OF NEURONS

In [103]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units',min_value=8,max_value=128,step=8)
    model.add(Dense(units=units, activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [104]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5 , directory = 'mydir'
                       , project_name = 'tuner')

In [105]:
tuner.search(x_train,y_train,epochs=5,validation_data = (x_test,y_test))


Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 06s


In [107]:
tuner.get_best_hyperparameters()[0].values

{'units': 72}

In [108]:
model = tuner.get_best_models(num_models=1)[0]

In [109]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 72)             │           648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            73 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 721 (2.82 KB)

 Trainable params: 721 (2.82 KB)

 Non-trainable params: 0 (0.00 B)

In [110]:
model.fit(x_train,y_train,batch_size=32,epochs=200,initial_epoch=6)

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8049 - loss: 0.4552  
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7681 - loss: 0.4928 
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7697 - loss: 0.4836 
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7912 - loss: 0.4677 
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.7677 - loss: 0.4826
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8027 - loss: 0.4575 
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7872 - loss: 0.4469 
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7846 - loss: 0.4473 
Epoch 15/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7853 - loss: 0.4591 
Epoch 16/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7854 - loss: 0.4672 
Epoch 17/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7885 - loss: 0.4565 
Epoch 18/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2

## HOW TO FIND RIGHT NUMBER OF LAYERS

In [111]:
def build_model(hp):
    model=Sequential()
    model.add(Dense(72,activation='relu',input_dim=8))
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [114]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='n_Layers',project_name='num_layers')

In [116]:
tuner.search(x_train,y_train,epochs=5,validation_data = (x_test,y_test))


Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 05s


In [119]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7}

In [121]:
model = tuner.get_best_models(num_models=1)[0]

/home/ashu/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 20 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [123]:
model.fit(x_train,y_train,initial_epoch=6,epochs=100,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7924 - loss: 0.4729 - val_accuracy: 0.7273 - val_loss: 0.5666
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7917 - loss: 0.4486 - val_accuracy: 0.7403 - val_loss: 0.5135
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7964 - loss: 0.4324 - val_accuracy: 0.7013 - val_loss: 0.5735
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8237 - loss: 0.3961 - val_accuracy: 0.7468 - val_loss: 0.5126
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8298 - loss: 0.3747 - val_accuracy: 0.7468 - val_loss: 0.5074
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8467 - loss: 0.3771 - val_accuracy: 0.7532 - val_loss: 0.5086
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8458 - loss: 0.3378 - val_accuracy: 0.7532 - val_loss: 0.5121
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8342 - loss: 0.3744 - val_accuracy: 0.77

## DOING ALL THE ABOVE THINGS AT ONCE

In [134]:
def build_model(hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        if counter == 0:
            model.add(Dense(hp.Int('units_' + str(i), min_value=8, max_value=128, step=8),
                            activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh', 'sigmoid']),
                            input_dim=8))
            model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(Dense(hp.Int('units_' + str(i), min_value=8, max_value=128, step=8),
                            activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh', 'sigmoid'])))
            model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter += 1
    model.add(Dense(1, activation='sigmoid'))
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta', 'nadam', 'adadelta'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [135]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3 , directory = 'mydir1'
                       , project_name = 'final1')

In [136]:
tuner.search(x_train,y_train,epochs=5,validation_data = (x_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 07s


In [137]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units_0': 64,
 'activation_0': 'tanh',
 'dropout0': 0.9,
 'optimizer': 'nadam',
 'units_1': 48,
 'activation_1': 'relu',
 'dropout1': 0.6,
 'units_2': 128,
 'activation_2': 'relu',
 'dropout2': 0.4,
 'units_3': 120,
 'activation_3': 'tanh',
 'dropout3': 0.3,
 'units_4': 48,
 'activation_4': 'relu',
 'dropout4': 0.6,
 'units_5': 40,
 'activation_5': 'sigmoid',
 'dropout5': 0.6,
 'units_6': 40,
 'activation_6': 'relu',
 'dropout6': 0.5}

In [138]:
model = tuner.get_best_models(num_models=1)[0]

/home/ashu/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 15 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [139]:
model.fit(x_train,y_train,initial_epoch=6,epochs=200,validation_data=(x_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6184 - loss: 0.7481 - val_accuracy: 0.7532 - val_loss: 0.5665
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6504 - loss: 0.6905 - val_accuracy: 0.7727 - val_loss: 0.5546
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6571 - loss: 0.6634 - val_accuracy: 0.7662 - val_loss: 0.5435
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5945 - loss: 0.7635 - val_accuracy: 0.7662 - val_loss: 0.5380
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6719 - loss: 0.6382 - val_accuracy: 0.7662 - val_loss: 0.5314
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6250 - loss: 0.6969 - val_accuracy: 0.7597 - val_loss: 0.5263
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6793 - loss: 0.6298 - val_accuracy: 0.7597 - val_loss: 0.5210
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6707 - loss: 0.6241 - val_accuracy: 0.75